## 1枚のCT画像をスペクトラルクラスタリング

### ライブラリのインポート

In [1]:
import numpy as np
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt
import sys
import datetime
import math
from skimage.draw import line
from multiprocessing import Pool
from concurrent.futures import ProcessPoolExecutor
from collections import defaultdict
import random
import pandas as pd
import networkx as nx

### ガボールフィルタ

In [2]:
def gabor(img,k,sigma,lam,gam):
    gabor = cv2.getGaborKernel((k,k), sigma, np.radians(0), lam, gam, 0)
    img_0 = cv2.filter2D(img, cv2.CV_32S, gabor,cv2.BORDER_CONSTANT)

    gabor = cv2.getGaborKernel((k,k), sigma, np.radians(10), lam, gam, 0)
    img_10 = cv2.filter2D(img, cv2.CV_32S, gabor,cv2.BORDER_CONSTANT)

    gabor = cv2.getGaborKernel((k,k), sigma, np.radians(20), lam, gam, 0)
    img_20 = cv2.filter2D(img, cv2.CV_32S, gabor,cv2.BORDER_CONSTANT)

    gabor = cv2.getGaborKernel((k,k), sigma, np.radians(30), lam, gam, 0)
    img_30 = cv2.filter2D(img, cv2.CV_32S, gabor,cv2.BORDER_CONSTANT)

    gabor = cv2.getGaborKernel((k,k), sigma, np.radians(40), lam, gam, 0)
    img_40 = cv2.filter2D(img, cv2.CV_32S, gabor,cv2.BORDER_CONSTANT)

    gabor = cv2.getGaborKernel((k,k), sigma, np.radians(50), lam, gam, 0)
    img_50 = cv2.filter2D(img, cv2.CV_32S, gabor,cv2.BORDER_CONSTANT)

    gabor = cv2.getGaborKernel((k,k), sigma, np.radians(60), lam, gam, 0)
    img_60 = cv2.filter2D(img, cv2.CV_32S, gabor,cv2.BORDER_CONSTANT)

    gabor = cv2.getGaborKernel((k,k), sigma, np.radians(70), lam, gam, 0)
    img_70 = cv2.filter2D(img, cv2.CV_32S, gabor,cv2.BORDER_CONSTANT)
    
    gabor = cv2.getGaborKernel((k,k), sigma, np.radians(80), lam, gam, 0)
    img_80 = cv2.filter2D(img, cv2.CV_32S, gabor,cv2.BORDER_CONSTANT)

    gabor = cv2.getGaborKernel((k,k), sigma, np.radians(90), lam, gam, 0)
    img_90 = cv2.filter2D(img, cv2.CV_32S, gabor,cv2.BORDER_CONSTANT)

    gabor = cv2.getGaborKernel((k,k), sigma, np.radians(100), lam, gam, 0)
    img_100 = cv2.filter2D(img, cv2.CV_32S, gabor,cv2.BORDER_CONSTANT)

    gabor = cv2.getGaborKernel((k,k), sigma, np.radians(110), lam, gam, 0)
    img_110 = cv2.filter2D(img, cv2.CV_32S, gabor,cv2.BORDER_CONSTANT)

    gabor = cv2.getGaborKernel((k,k), sigma, np.radians(120), lam, gam, 0)
    img_120 = cv2.filter2D(img, cv2.CV_32S, gabor,cv2.BORDER_CONSTANT)
    
    gabor = cv2.getGaborKernel((k,k), sigma, np.radians(130), lam, gam, 0)
    img_130 = cv2.filter2D(img, cv2.CV_32S, gabor,cv2.BORDER_CONSTANT)

    gabor = cv2.getGaborKernel((k,k), sigma, np.radians(140), lam, gam, 0)
    img_140 = cv2.filter2D(img, cv2.CV_32S, gabor,cv2.BORDER_CONSTANT)

    gabor = cv2.getGaborKernel((k,k), sigma, np.radians(150), lam, gam, 0)
    img_150 = cv2.filter2D(img, cv2.CV_32S, gabor,cv2.BORDER_CONSTANT)

    gabor = cv2.getGaborKernel((k,k), sigma, np.radians(160), lam, gam, 0)
    img_160 = cv2.filter2D(img, cv2.CV_32S, gabor,cv2.BORDER_CONSTANT)

    gabor = cv2.getGaborKernel((k,k), sigma, np.radians(170), lam, gam, 0)
    img_170 = cv2.filter2D(img, cv2.CV_32S, gabor,cv2.BORDER_CONSTANT)
    
    return img_0,img_10,img_20,img_30,img_40,img_50,img_60,img_70,img_80,img_90,img_100,img_110,img_120,img_130,img_140,img_150,img_160,img_170

In [13]:
#def spectral_clusering(img_num):
#num_list = [666,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730]

#for img_num in range(545,530,-1):
#def spectral_clustering(img_num):
img_num = 640
print('-----------------------------------------')
print(img_num)

#ノード数カウント用
cnt = 1

#クラスタ分割数
k_c = 200

now = datetime.datetime.now()
print(now)
print('ノード番号付与中')

#入力画像を取得
img = cv2.imread('../../flower_CT_photo/ORA/[vg-data] ORA/volume_1/ORA-{0:03d}.tif'.format(img_num),0)
#画像のサイズを取得
height,width = img.shape

#入力画像にノンローカルミーンフィルタを適用し2値化
img_NLMD = cv2.fastNlMeansDenoising(img,h=6)
print(img_NLMD.shape)
th_img = np.where(img_NLMD>50 ,255,0)

cv2.imwrite('Output/Spectral_sys/99_NLMD-h6/ORA-{0:03d}_NLMD-h6.png'.format(img_num),img_NLMD)

#ノード番号を格納する配列を定義
temp = np.zeros((height,width),dtype = 'i4')

G = nx.Graph()
#pos = {}
edge_weight = []

#閾値以上のピクセルにノード番号付与
for y in tqdm(range(height)):
    for x in range(width):
        if(th_img[y,x]==255):
            temp[y,x] = cnt
            G.add_node(cnt-1)
            cnt += 1

now = datetime.datetime.now()
print(now)
print('隣接行列を宣言')

#スパース行列を使うときは以下
#A = lil_matrix((cnt-1,cnt-1),dtype='float')
A = np.zeros((cnt-1,cnt-1))

now = datetime.datetime.now()
print(now)
print('ガボールフィルタ')

#パラメータ設定
k = 30
sigma = 1.11
lam = 10
gam = 0.09
img_0,img_10,img_20,img_30,img_40,img_50,img_60,img_70,img_80,img_90,img_100,\
img_110,img_120,img_130,img_140,img_150,img_160,img_170 = gabor(img_NLMD,k,sigma,lam,gam)

now = datetime.datetime.now()
print(now)
print('隣接行列作成中')

r_th = 10
a = 0.001
b = 0.001

for y in tqdm(range(50,height-50)):
    for x in range(50,width-50):

        if(temp[y,x]>0):
            node1 = temp[y,x]-1
            for y_t in range(y,y+10):
                if(y == y_t):
                    for x_t in range(x,x+10):
                        if((temp[y_t,x_t]>0) and (x != x_t)):
                            node2 = temp[y_t,x_t]-1
                            r = math.sqrt((x-x_t)**2 + (y-y_t)**2)

                            esc = 0
                            flag = 0
                            i = 0

                            rr,cc = line(y,x,y_t,x_t)

                            while(esc==0):
                                if(th_img[rr[i],cc[i]]==0):
                                    flag = 1
                                    esc = 1
                                elif(i < rr.shape[0]-1):
                                    i += 1
                                else:
                                    esc = 1

                            if(flag==0):

                                d = math.sqrt((img_0[y,x]-img_0[y_t,x_t])**2+(img_10[y,x]-img_10[y_t,x_t])**2+(img_20[y,x]-img_20[y_t,x_t])**2+\
                                              (img_30[y,x]-img_30[y_t,x_t])**2+(img_40[y,x]-img_40[y_t,x_t])**2+(img_50[y,x]-img_50[y_t,x_t])**2+\
                                              (img_60[y,x]-img_60[y_t,x_t])**2+(img_70[y,x]-img_70[y_t,x_t])**2+(img_80[y,x]-img_80[y_t,x_t])**2+\
                                              (img_90[y,x]-img_90[y_t,x_t])**2+(img_100[y,x]-img_100[y_t,x_t])**2+(img_110[y,x]-img_110[y_t,x_t])**2+\
                                              (img_120[y,x]-img_120[y_t,x_t])**2+(img_130[y,x]-img_130[y_t,x_t])**2+(img_140[y,x]-img_140[y_t,x_t])**2+\
                                              (img_150[y,x]-img_150[y_t,x_t])**2+(img_160[y,x]-img_160[y_t,x_t])**2)+(img_170[y,x]-img_170[y_t,x_t])**2

                                W = math.exp(-a*r)+math.exp(-b*d)

                                A[node1,node2] = W
                                A[node2,node1] = W

                                G.add_edge(node1,node2,weight=W)
                                    
                else:
                    for x_t in range(x-10,x+10):
                        if(temp[y_t,x_t]>0):
                            node2 = temp[y_t,x_t]-1
                            r = math.sqrt((x-x_t)**2 + (y-y_t)**2)
                            
                            esc = 0
                            flag = 0
                            i = 0


                            rr,cc = line(y,x,y_t,x_t)

                            while(esc==0):
                                if(th_img[rr[i],cc[i]]==0):
                                    flag = 1
                                    esc = 1
                                elif(i < rr.shape[0]-1):
                                    i += 1
                                else:
                                    esc = 1


                            if(flag==0):
                                d = math.sqrt((img_0[y,x]-img_0[y_t,x_t])**2+(img_10[y,x]-img_10[y_t,x_t])**2+(img_20[y,x]-img_20[y_t,x_t])**2+\
                                              (img_30[y,x]-img_30[y_t,x_t])**2+(img_40[y,x]-img_40[y_t,x_t])**2+(img_50[y,x]-img_50[y_t,x_t])**2+\
                                              (img_60[y,x]-img_60[y_t,x_t])**2+(img_70[y,x]-img_70[y_t,x_t])**2+(img_80[y,x]-img_80[y_t,x_t])**2+\
                                              (img_90[y,x]-img_90[y_t,x_t])**2+(img_100[y,x]-img_100[y_t,x_t])**2+(img_110[y,x]-img_110[y_t,x_t])**2+\
                                              (img_120[y,x]-img_120[y_t,x_t])**2+(img_130[y,x]-img_130[y_t,x_t])**2+(img_140[y,x]-img_140[y_t,x_t])**2+\
                                              (img_150[y,x]-img_150[y_t,x_t])**2+(img_160[y,x]-img_160[y_t,x_t])**2)+(img_170[y,x]-img_170[y_t,x_t])**2

                                W = math.exp(-a*r)+math.exp(-b*d)

                                A[node1,node2] = W
                                A[node2,node1] = W

                                G.add_edge(node1,node2,weight=W)

-----------------------------------------
640
2022-01-03 18:42:20.997105
ノード番号付与中
(858, 915)


100%|█████████████████████████████████████████████████████| 858/858 [00:00<00:00, 1631.78it/s]


2022-01-03 18:42:27.607386
隣接行列を宣言
2022-01-03 18:42:28.079896
ガボールフィルタ
2022-01-03 18:42:28.551982
隣接行列作成中


100%|███████████████████████████████████████████████████████| 758/758 [13:26<00:00,  1.06s/it]


In [14]:
df = nx.to_pandas_edgelist(G)

In [16]:
df

,source,target,weight
0,0,1,1.025190
1,0,2,0.998112
2,0,3,0.997004
3,0,4,0.996008
4,0,5,0.995012
...,...,...,...
5479648,69864,69868,0.993936
5479649,69864,69869,0.994914
5479650,69865,69866,0.990985
5479651,69865,69867,0.991970


In [15]:
df.to_csv("test_640.csv",header=False, index=False)

In [12]:
G

In [7]:
cnt

129271